# **_Stock Market Trend Analysis (2021–2025)_**
### Group Members:
- Farhan Haider
- Imran Khan
- Saad Farooq
****

### **Datasets:** Companies Listing.csv, Pakistan Stock Market Data 2021-2025.csv

In [ ]:
import pandas as pd, numpy as np, math, os

In [ ]:
import matplotlib.pyplot as plt, seaborn as sns

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm

plt.rcParams['figure.figsize'] = (12,5)
sns.set_theme(style="darkgrid")